from google.colab import drive

drive.mount('/content/drive')

!pip install igraph

In [2]:
import pickle
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

## Besprechung 2 (Notes) 

sind 2 claims gegen oder für einander ? claim mit Tiefe (Diskussion mit versch. Tiefen - Normalisierung ? da tiefe 4 bei einem task fein granulierter als 10 in einem anderen) Sind 2 claims auf dem selben Level ?

kann das modell verschiedene tiefenebenen erkennen ? shortest path? 2 claims mit distanz 2 : sind sie für oder gegneinander: PROBE dynamich gestalten, wir können auch probe mit länge 3, 4 // je weiter auseinander je mehr anders (generell) nicht immer.

von gleicher diskussion oder nicht ?

2-3 claims + base node // sind nodes/claims antworten oder nicht ?

start oder endclaim ? oder basisclaim?

csv - format (speziell) Daten abspeichern, das kann nachher direkt dem Classifier trainiert werden und reporten.

Probing: Visualisieren --- visio/konzeptionell > input , von wo kommt das label / black box

In [2]:
pickle_file_path = "../../../data_preparation/kialo_data/3_parsed_node_data/parsed_discussion_graphs.pickle"
with open(pickle_file_path, 'rb') as f:
    parsed_discussion_graphs = pickle.load(f)

General ID-Structure: [DiscussionID . ClaimID], claim_ids are not always in chronological order (so it not always follows 0.2 on 0.1)
Some ID´s are also skipped / missing / deleted


In [3]:
print("total number of discussion graphs ", len(parsed_discussion_graphs))

total number of discussion graphs  17832


In [4]:
print("First Discussion: Overview", parsed_discussion_graphs[0])


First Discussion: Overview IGRAPH UN-- 601 610 --
+ attr: name (v), relation (e)
+ edges (vertex names):
{'id': '333.1', 'text': 'An anonymous untrackable digital currency \\(ADC\\),
like Bitcoin, is beneficial for civilized societies.'}--{'id': '333.2',
'text': "Cash is a necessary part of any functional society's economic model
and cryptocurrencies are an easier and online-ready form of cash. They are the
natural progression of what money is becoming in the digital age."}, {'id':
'333.1', 'text': 'An anonymous untrackable digital currency \\(ADC\\), like
Bitcoin, is beneficial for civilized societies.'}--{'id': '333.3', 'text':
'Most ordinary citizens would not use such a currency anyway, rendering all
potential benefits moot.'}, {'id': '333.3', 'text': 'Most ordinary citizens
would not use such a currency anyway, rendering all potential benefits
moot.'}--{'id': '333.7', 'text': 'If people really wanted anonymity, they
would use more cash and make less use of credit and debit cards.'

In [5]:
print("each discussion is a sequential igraph object storing edges (relations) and nodes (content) per claim: ",
      parsed_discussion_graphs[0].vs)

each discussion is a sequential igraph object storing edges (relations) and nodes (content) per claim:  <igraph.seq.VertexSeq object at 0x000001F4CA8670B0>


## Claim Level

Claims are accessed on the igraph object by the ".vs" - attribute.

For each claim it is possible to access its node(claim id and content) and its edges (relations)

For each claim-node 2 attributes are stored: 
- id : the claims id
- text : the claims content


In [6]:
print(
    "to see the objects more clearly let´s print the first discussion´s nodes in readable format with the help of the name attribute (selected number=",
    len(parsed_discussion_graphs[0].vs["name"][0:20]), ") : \n\n",
    parsed_discussion_graphs[0].vs["name"][0:20])

to see the objects more clearly let´s print the first discussion´s nodes in readable format with the help of the name attribute (selected number= 20 ) : 

 [{'id': '333.0', 'text': 'Anonymous currency discussions'}, {'id': '333.1', 'text': 'An anonymous untrackable digital currency \\(ADC\\), like Bitcoin, is beneficial for civilized societies.'}, {'id': '333.2', 'text': "Cash is a necessary part of any functional society's economic model and cryptocurrencies are an easier and online-ready form of cash. They are the natural progression of what money is becoming in the digital age."}, {'id': '333.3', 'text': 'Most ordinary citizens would not use such a currency anyway, rendering all potential benefits moot.'}, {'id': '333.4', 'text': 'Most people pay taxes and banks report all their in/outflows above certain limits.'}, {'id': '333.5', 'text': 'Cryptocurrencies are not like cash.'}, {'id': '333.7', 'text': 'If people really wanted anonymity, they would use more cash and make less use of 

In [7]:
# accessing id
print(parsed_discussion_graphs[0].vs["name"][0]["id"])

333.0


In [8]:
# accessing text
print(parsed_discussion_graphs[0].vs["name"][0]["text"])

Anonymous currency discussions


For each edge (relation) we can access its target and source nodes , its relation type and its edge_id.
One edge is always a relation between 2 claims.

In [9]:
edge_example = parsed_discussion_graphs[0].es[0]
print("1st edge (edge 0) ", edge_example)
print("2nd edge (edge 1) ", parsed_discussion_graphs[0].es[1])

1st edge (edge 0)  igraph.Edge(<igraph.Graph object at 0x000001F4C6A1EB40>, 0, {'relation': 1})
2nd edge (edge 1)  igraph.Edge(<igraph.Graph object at 0x000001F4C6A1EB40>, 1, {'relation': -1})


In [10]:
print("The type of the 1st relation. 1 is pro and -1 is con:", parsed_discussion_graphs[0].es[0]["relation"])

The type of the 1st relation. 1 is pro and -1 is con: 1


In [11]:
# the node/vertex, where the relation comes from
print("the node/claim, from where the 1st relation comes from:", edge_example.source_vertex)
print()
print("access only claim_id of the source_claim (so claim id of 222.5 is 5): ", edge_example.source)
print("access full id of the source claim:", edge_example.source_vertex["name"]["id"])

the node/claim, from where the 1st relation comes from: igraph.Vertex(<igraph.Graph object at 0x000001F4C6A1EB40>, 1, {'name': {'id': '333.1', 'text': 'An anonymous untrackable digital currency \\(ADC\\), like Bitcoin, is beneficial for civilized societies.'}})

access only claim_id of the source_claim (so claim id of 222.5 is 5):  1
access full id of the source claim: 333.1


In [12]:
''  # the node where the relation goes to
print("the node, to where the 1st relation goes to ", edge_example.target_vertex)
print()
print("claim id of the target claim: ", edge_example.target)
print("access full id of the target claim:", edge_example.source_vertex["name"]["id"])

the node, to where the 1st relation goes to  igraph.Vertex(<igraph.Graph object at 0x000001F4C6A1EB40>, 2, {'name': {'id': '333.2', 'text': "Cash is a necessary part of any functional society's economic model and cryptocurrencies are an easier and online-ready form of cash. They are the natural progression of what money is becoming in the digital age."}})

claim id of the target claim:  2
access full id of the target claim: 333.1


In [13]:
print("access claim_id tuple of source and target claim (source,target): ", edge_example.tuple)

access claim_id tuple of source and target claim (source,target):  (1, 2)


## Removing empty nodes (v2)

### Further investigating the .0 claim id

- already known: a lot of claims have the text 'Please enter the multiple choice question.' as their first claim (in .0 position) and this claim can be removed since it is not a real claim
- however some exceptions arise, which mustn´t be deleted

In [14]:
{'id': '444.0', 'text': 'Please enter the multiple choice question.'}

{'id': '444.0', 'text': 'Please enter the multiple choice question.'}

In [15]:
noise_start_claim_texts = []
errors = []

In [16]:
for i in range(len(parsed_discussion_graphs)):
    graph = parsed_discussion_graphs[i]

    if graph is not None:
        if hasattr(graph, 'vs') and len(graph.vs) > 0:
            try:
                if graph.vs["name"][0]["text"] == 'Please enter the multiple choice question.' or graph.vs["name"][0][
                    "text"] == '' or graph.vs["name"][0]["text"] is None:
                    print(graph.vs["name"][0]["text"])
                    print("--------------------------------------------------")
                    noise_start_claim_texts.append((graph.vs["name"][0]["id"],
                                                    graph.vs["name"][0]["text"],
                                                    graph))
            except KeyError:
                print(f"KeyError encountered for graph at index {i}")
                errors.append((parsed_discussion_graphs[i], "KeyError"))
        else:
            print(f"Graph at index {i} has no vertices.")
            errors.append((parsed_discussion_graphs[i], "has no vertices"))
    else:
        print(f"Graph at index {i} is None.")
        errors.append((parsed_discussion_graphs[i], "None"))


Please enter the multiple choice question.
--------------------------------------------------
Please enter the multiple choice question.
--------------------------------------------------
Please enter the multiple choice question.
--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

-----------------------------------------

In [17]:
print(
    "Number of discussions with noise ('' or Multiple Choice Question) at position vs[""name""][0] for claim content and claim id: ",
    len(noise_start_claim_texts))
print("Number of discussions with Errors (KeyError,None,No Vertices): ", len(errors))

Number of discussions with noise ('' or Multiple Choice Question) at position vs[name][0] for claim content and claim id:  14637
Number of discussions with Errors (KeyError,None,No Vertices):  851


In [18]:
#print(parsed_discussion_graphs[402].vs["name"][0]["id"])
print("content of base claim (noise): ")
parsed_discussion_graphs[402].vs["name"][0]["text"]
print("content of 1st claim (noise): ")
parsed_discussion_graphs[402].vs["name"][1]["text"]

content of base claim (noise): 
content of 1st claim (noise): 


'The primary purpose of marriage is procreating and raising children.'

In [19]:
no_vertices_example = parsed_discussion_graphs[17766].vs
print(no_vertices_example)
print(no_vertices_example["name"])
# will throw error cause its empty: 
print(no_vertices_example["name"][600])

[]


IndexError: list index out of range

since graphs in errors folder have no further use, all graphs in errors can be entirely removed

for the noisy .0 positions we just remove this position

In [20]:
# Length before cleaning
original_len_graphs = len(parsed_discussion_graphs)
original_len_nodes = sum(len(graph.vs) if graph is not None else 0 for graph in parsed_discussion_graphs)
print(original_len_nodes, original_len_graphs)

631841 17832


In [21]:
import pickle

# Step 1: Identify IDs and Indices to Remove
ids_to_remove = set([entry[0] for entry in noise_start_claim_texts])
indices_to_remove = set([parsed_discussion_graphs.index(entry[0]) for entry in errors if entry[0] is not None])

# Step 2: Iterate Through Graphs and Remove Noise and Errors
cleaned_graphs = []
for i, graph in enumerate(parsed_discussion_graphs):
    if i in indices_to_remove:
        continue  # Skip graphs that are in the errors list
    if graph is not None:
        if hasattr(graph, 'vs') and len(graph.vs) > 0:
            try:
                node_id = graph.vs["name"][0]["id"]
                if node_id in ids_to_remove:
                    graph.delete_vertices(0)
                cleaned_graphs.append(graph)
            except KeyError:
                continue

# Step 3: Save Cleaned Data
with open("cleaned_parsed_discussion_graphs.pkl", "wb") as f:
    pickle.dump(cleaned_graphs, f)


In [22]:
# Length after cleaning
cleaned_len_graphs = len(cleaned_graphs)
cleaned_len_nodes = sum(len(graph.vs) for graph in cleaned_graphs)

# Calculate the number of removed graphs and nodes
removed_graphs = original_len_graphs - cleaned_len_graphs
removed_nodes = original_len_nodes - cleaned_len_nodes

print(f"Removed graphs: {removed_graphs}, should be around 800")
print(f"Removed nodes: {removed_nodes}, should be around 14000")

Removed graphs: 851, should be around 800
Removed nodes: 14637, should be around 14000


In [23]:
# Initialize a list to keep track of cleaned graphs
cleaned_graphs_v2 = []

# Traverse through each graph in the cleaned_graphs list
for graph in cleaned_graphs:
    if graph is None:
        cleaned_graphs_v2.append(None)
        continue

    # Identify vertices to remove based on empty or None 'id' and 'text'
    remove_indices = [v.index for v in graph.vs if
                      not graph.vs["name"][v.index]["id"] or graph.vs["name"][v.index]["id"] == '' or
                      not graph.vs["name"][v.index]["text"] or graph.vs["name"][v.index]["text"] == '']

    # Create a new graph with the vertices removed
    new_graph = graph.subgraph([v.index for v in graph.vs if v.index not in remove_indices])

    cleaned_graphs_v2.append(new_graph)

# Now cleaned_graphs_v2 contains graphs with empty or None 'id' and 'text' vertices removed
print("number of empty nodes removed):", len(cleaned_graphs_v2))


number of empty nodes removed): 16981


#Save the cleaned_graphs_v2 list to a pickle file for future use
import pickle

#Specify the path where the pickle file will be saved
cleaned_graphs_path_v2 = "cleaned_graphs_v2.pkl"

#Open a file and save the pickle
with open(cleaned_graphs_path_v2, "wb") as f:
    pickle.dump(cleaned_graphs_v2, f)

cleaned_graphs_path_v2


In [25]:
remove_indices

[]

## Probing 1 Data Preparation (TO BE CONTINUED)

In [25]:
import random


# Check before accessing vertex data
def safe_vertex_access(graph, vertex_index, field):
    try:
        return graph.vs[vertex_index]["name"].get(field, None)
    except IndexError:
        return None


# Sample code to simulate graph traversal based on the strategies discussed.
# This code assumes that `cleaned_graphs_v2` is a list of igraph objects.
def random_traversal(cleaned_graphs):
    """"starts at base claim and randomly traverses the graph 2 times to a certain randomly chosen depth within the max depth"""
    #MIN_DEPTH_THRESHOLD = 2  # Set a minimum depth
    for graph_index, graph in enumerate(cleaned_graphs):
        #if graph is None:
        #continue
        base_claim_index = safe_vertex_access(graph, 0, "id")
        base_claim_text = safe_vertex_access(graph, 0, "text")

        if base_claim_index is None or base_claim_text is None:
            print(f"Skipping graph {graph_index} due to missing vertex or attribute.")
            continue  # Skip this graph

        # Initialize
        # Start from the base claim
        base_claim_index = graph.vs["name"][0]["id"]
        base_claim_text = graph.vs["name"][0]["text"]
        print("Debug: base claim index:", base_claim_index, "base claim text:", base_claim_text)
        visited_nodes = set()
        visited_nodes.add(base_claim_index)  # Base claim is always visited first

        # Dynamic Max Depth
        max_depth = graph.diameter() // 2  # max_depth = max(graph.diameter() // 2, MIN_DEPTH_THRESHOLD)
        if max_depth < 1:
            max_depth = 1  # or some other default value

        # Debugging Print
        print(f"Debug - Number of vertices: {len(graph.vs)}, Max Depth: {max_depth}")

        # Random Selection of Depth Limit
        try:
            random_depth_limit = random.randint(1, max_depth)
            print("Debug - Random Depth Limit:", random_depth_limit)
        except ValueError:
            print("Skipping graph due to invalid max_depth")
            continue

        # Store traversal data for two claims
        traversal_data = []

        for _ in range(2):  # for two random claims
            current_node = base_claim_index
            current_depth = 0
            current_stance = 1  # Start with neutral stance

            while current_depth < random_depth_limit:
                # Get neighbors that haven't been visited
                neighbors = [n for n in graph.neighbors(current_node) if n not in visited_nodes]

                if not neighbors:
                    break  # Exit if there are no unvisited neighbors

                next_node = random.choice(neighbors)
                visited_nodes.add(next_node)

                # Update stance
                edge_relation = graph.es[graph.get_eid(current_node, next_node)]["relation"]
                current_stance *= edge_relation

                # Move to the next node
                current_node = next_node
                current_depth += 1

            traversal_data.append({
                "Claim ID": graph.vs[current_node]["name"]["id"],
                "Claim Text": graph.vs[current_node]["name"]["text"],
                "Stance": current_stance
            })

        if len(traversal_data) == 2:
            print(f"Traversal Data: {traversal_data}")
        else:
            print("Insufficient data for this graph.")

        return traversal_data

#random traversal round 1 with data, that still contains spam messages
random_traversal(cleaned_graphs_v2)

In [21]:
cleaned_graphs_path_v3 = "cleaned_graphs_v3.pkl"
with open(cleaned_graphs_path_v3, 'rb') as f:
    cleaned_graphs_v3 = pickle.load(f)


In [31]:
# for efficiency reasons we convert ids to numbers (float) 
print("claim id: ", type(cleaned_graphs_v3[0].vs["name"][0]["id"]), "edges : ", type(cleaned_graphs_v3[0].es[0]),
      " relation type : ", type(cleaned_graphs_v3[0].es[0]["relation"]), "source vertex id: ",
      type(cleaned_graphs_v3[0].es[0].source_vertex["name"]["id"]), "target vertex id:  ",
      type(cleaned_graphs_v3[0].es[0].target_vertex["name"]["id"]))

claim id:  <class 'str'> edges :  <class 'igraph.Edge'>  relation type :  <class 'int'> source vertex id:  <class 'str'> target vertex id:   <class 'str'>


In [32]:
def convert_ids_to_float(graphs):
    for graph in graphs:
        # Convert vertex IDs
        for vertex in graph.vs:
            vertex["name"]["id"] = float(vertex["name"]["id"])

        # Convert edge source and target IDs
        for edge in graph.es:
            source_idx = edge.source
            target_idx = edge.target
            graph.vs[source_idx]["name"]["id"] = float(graph.vs[source_idx]["name"]["id"])
            graph.vs[target_idx]["name"]["id"] = float(graph.vs[target_idx]["name"]["id"])
    return graphs


# Assuming cleaned_graphs_v2 contains your list of graphs
cleaned_graphs_v4 = convert_ids_to_float(cleaned_graphs_v3)

In [33]:

print("claim id: ", type(cleaned_graphs_v4[0].vs["name"][0]["id"]), "edges : ", type(cleaned_graphs_v4[0].es[0]),
      " relation type : ", type(cleaned_graphs_v4[0].es[0]["relation"]), "source vertex id: ",
      type(cleaned_graphs_v4[0].es[0].source_vertex["name"]["id"]), "target vertex id:  ",
      type(cleaned_graphs_v4[0].es[0].target_vertex["name"]["id"]))

claim id:  <class 'float'> edges :  <class 'igraph.Edge'>  relation type :  <class 'int'> source vertex id:  <class 'float'> target vertex id:   <class 'float'>


In [44]:
print("claim 1 with id = ", cleaned_graphs_v4[0].vs["name"][1]["id"], "has the following target vertex: ",
      cleaned_graphs_v4[0].es[1].target_vertex["name"]["id"],
      "it can be verified that the source vertex to this target vertex is the claim 1 with id = ",
      cleaned_graphs_v4[0].es[1].source_vertex["name"]["id"])
#cleaned_graphs_v4[0].es[1]

claim 1 with id =  333.1 has the following target vertex:  333.3 it can be verified that the source vertex to this target vertex is the claim 1 with id =  333.1


In [50]:
"total number of edges in graph/discussion 0", len(cleaned_graphs_v4[0].es)


('total number of edges in graph/discussion 0', 603)

In [56]:
cleaned_graphs_v4[0].vs["name"][1:]

[{'id': 333.1,
  'text': 'An anonymous untrackable digital currency \\(ADC\\), like Bitcoin, is beneficial for civilized societies.'},
 {'id': 333.2,
  'text': "Cash is a necessary part of any functional society's economic model and cryptocurrencies are an easier and online-ready form of cash. They are the natural progression of what money is becoming in the digital age."},
 {'id': 333.3,
  'text': 'Most ordinary citizens would not use such a currency anyway, rendering all potential benefits moot.'},
 {'id': 333.4,
  'text': 'Most people pay taxes and banks report all their in/outflows above certain limits.'},
 {'id': 333.5, 'text': 'Cryptocurrencies are not like cash.'},
 {'id': 333.7,
  'text': 'If people really wanted anonymity, they would use more cash and make less use of credit and debit cards.'},
 {'id': 333.9, 'text': 'Cryptocurrencies make money laundering much easier.'},
 {'id': 333.1,
  'text': 'Civilized societies try to exterminate money laundering.'},
 {'id': 333.11,
  't

In [62]:
cleaned_graphs_v4[0].es[1].target_vertex["name"][1]["id"]

KeyError: 'name'

In [74]:
example_graph = cleaned_graphs_v4[0]

In [75]:
example_graph_edges = example_graph.es
example_graph_nodes = example_graph.vs
print(example_graph_edges, example_graph_nodes)

<igraph.seq.EdgeSeq object at 0x0000023266C2EE00> <igraph.seq.VertexSeq object at 0x0000023260292890>


In [79]:
for i, node in enumerate(example_graph_nodes):
    print("source: ", node["name"]["id"])
    print("target: ", example_graph_edges[i].target_vertex["name"]["id"])
    print("-----------------------------------")


source:  333.0
target:  333.2
-----------------------------------
source:  333.1
target:  333.3
-----------------------------------
source:  333.2
target:  333.7
-----------------------------------
source:  333.3
target:  333.12
-----------------------------------
source:  333.4
target:  333.17
-----------------------------------
source:  333.5
target:  333.18
-----------------------------------
source:  333.7
target:  333.19
-----------------------------------
source:  333.9
target:  333.2
-----------------------------------
source:  333.1
target:  333.21
-----------------------------------
source:  333.11
target:  333.22
-----------------------------------
source:  333.12
target:  333.24
-----------------------------------
source:  333.14
target:  333.26
-----------------------------------
source:  333.15
target:  333.31
-----------------------------------
source:  333.16
target:  333.35
-----------------------------------
source:  333.17
target:  333.37
-----------------------------

In [73]:
cleaned_graphs_v4[0].es[
    1].target_vertex

igraph.Vertex(<igraph.Graph object at 0x00000232607EF140>, 3, {'name': {'id': 333.3, 'text': 'Most ordinary citizens would not use such a currency anyway, rendering all potential benefits moot.'}})

In [72]:
for i, node in enumerate(cleaned_graphs_v4[0].vs["name"][1:]):  #exclude base claim at position 0
    if len(cleaned_graphs_v4[0].es[
               i].target_vertex) > 1:  # checking for amount of attirbutes in target vertex - not what were asing for
        print("impossible")
        for j in range(len(cleaned_graphs_v4[0].es[i].target_vertex)):
            print(cleaned_graphs_v4[0].es[i].target_vertex["name"])
            #print(i, j)

In [57]:
# identify claims with more than one target vertex
for i, node in enumerate(cleaned_graphs_v4[0].vs["name"][1:]):  #exclude base claim at position 0
    if len(cleaned_graphs_v4[0].es[i].target_vertex["name"]) > 1:
        for j in range(len(cleaned_graphs_v4[0].es[i].target_vertex["name"])):
            print("target claim number:", j, cleaned_graphs_v4[0].es[i].target_vertex["name"][j]["id"])
            print("source claim: ", cleaned_graphs_v4[0].es[i].source_vertex["name"][j]["id"])



KeyError: 0

In [52]:
"all nodes/claims in graph/discussion 0", cleaned_graphs_v4[0].vs["name"]

[{'id': 333.0, 'text': 'Anonymous currency discussions'},
 {'id': 333.1,
  'text': 'An anonymous untrackable digital currency \\(ADC\\), like Bitcoin, is beneficial for civilized societies.'},
 {'id': 333.2,
  'text': "Cash is a necessary part of any functional society's economic model and cryptocurrencies are an easier and online-ready form of cash. They are the natural progression of what money is becoming in the digital age."},
 {'id': 333.3,
  'text': 'Most ordinary citizens would not use such a currency anyway, rendering all potential benefits moot.'},
 {'id': 333.4,
  'text': 'Most people pay taxes and banks report all their in/outflows above certain limits.'},
 {'id': 333.5, 'text': 'Cryptocurrencies are not like cash.'},
 {'id': 333.7,
  'text': 'If people really wanted anonymity, they would use more cash and make less use of credit and debit cards.'},
 {'id': 333.9, 'text': 'Cryptocurrencies make money laundering much easier.'},
 {'id': 333.1,
  'text': 'Civilized societies tr

In [51]:
cleaned_graphs_v4[0].es[1].target_vertex["name"]["id"]

333.3

In [36]:
cleaned_graphs_v4[0].es[0].target_vertex["name"]["id"]

333.2

In [26]:
#= random.choice(cleaned_graphs_v3)
traversal_data = random_traversal(cleaned_graphs_v3)

Debug: base claim index: 333.0 base claim text: Anonymous currency discussions
Debug - Number of vertices: 592, Max Depth: 7
Debug - Random Depth Limit: 7


RuntimeError: error while indexing vertex names; did you accidentally try to use a non-hashable object as a vertex name earlier? Check the name of vertex 591 ({'id': '333.1698', 'text': "Central Bank Digital Cryptocurrencies give agents of the state complete visibility and control over users' personal finances."})

In [39]:
# Assuming traversal_data is a list containing two dictionaries
# Each dictionary has "Claim ID", "Claim Text", and "Stance"

# Extract claims and their stances
claim1, stance1 = traversal_data[0]["Claim Text"], traversal_data[0]["Stance"]
claim2, stance2 = traversal_data[1]["Claim Text"], traversal_data[1]["Stance"]

# Determine the relationship between claim1 and claim2
# 1 if they are Pro to each other, -1 if they are Con to each other
relation_c1_to_c2 = 1 if stance1 == stance2 else -1

# Create X and y
X = [claim1, claim2]
y = [relation_c1_to_c2]

print("Training Example:")
print("X =", X)
print("y =", y)


Training Example:
X = ['Freeoracledumps.com | Get 1z0-997-20 Test Questions With Money Back Guarantee', 'Freeoracledumps.com | Get 1z0-997-20 Test Questions With Money Back Guarantee']
y = [1]


In [ ]:
import random


def traverse_from_base(graph):
    visited = set()
    traversal_data = []

    # Start from the base claim
    base_claim_id = graph.vs["name"][0]["id"]
    base_claim_text = graph.vs["name"][0]["text"]
    visited.add(base_claim_id)

    # Dynamically set max depth as graph size
    max_depth = len(graph.vs)

    # Randomly select a depth limit within the max depth
    depth_limit = random.randint(1, max_depth)

    # Initialize queue for BFS traversal
    queue = [(base_claim_id, 0, 1)]  # (claim_id, depth, stance)

    while queue:
        current_claim_id, current_depth, current_stance = queue.pop(0)

        # If we reached the depth limit, stop traversing
        if current_depth == depth_limit:
            break

        # Find edges where the source is the current claim
        for edge in graph.es.select(
                _source=graph.vs.find(lambda vertex: vertex["name"]["id"] == current_claim_id).index):
            target_vertex = edge.target_vertex
            target_claim_id = target_vertex["name"]["id"]
            relation = edge["relation"]

            if target_claim_id not in visited:
                visited.add(target_claim_id)
                new_stance = current_stance * relation  # Update the stance
                traversal_data.append((target_claim_id, target_vertex["name"]["text"], new_stance))

                # Add to queue for further traversal
                queue.append((target_claim_id, current_depth + 1, new_stance))

    return traversal_data


# Test the function on a random graph from cleaned_graphs_v2
random_graph = random.choice(cleaned_graphs_v2)
traversal_data = traverse_from_base(random_graph)

# Display the results
print("Traversal Data:")
for claim_id, claim_text, stance in traversal_data:
    print(f"Claim ID: {claim_id}, Claim Text: {claim_text}, Stance: {stance}")


In [ ]:
import random
import igraph as ig


# Function to calculate max depth from the base claim
def calculate_max_depth(graph, base_claim_index=0):
    return max([len(path) for path in graph.get_all_shortest_paths(base_claim_index)])


# Main function for traversal
def traverse_from_base(graph):
    visited_nodes = set()
    traversal_data = []

    base_claim_index = 0  # Assuming base claim is at index 0
    visited_nodes.add(base_claim_index)
    max_depth = calculate_max_depth(graph, base_claim_index)

    for _ in range(2):  # Collecting data for two different claims
        current_node = base_claim_index
        selected_depth = random.randint(1, max_depth)

        stance_list = []
        for depth in range(selected_depth):
            neighbors = [t for s, t in graph.get_edgelist() if s == current_node and t not in visited_nodes]

            if not neighbors:
                break

            next_node = random.choice(neighbors)
            edge = graph.es.find(_source=current_node, _target=next_node)

            visited_nodes.add(next_node)
            stance_list.append(edge['relation'])

            current_node = next_node

        claim_text = graph.vs[current_node]['name']['text']
        claim_id = graph.vs[current_node]['name']['id']

        # Determine overall stance towards the base claim
        overall_stance = 1 if sum(stance_list) > 0 else -1
        traversal_data.append({
            'Claim ID': claim_id,
            'Claim Text': claim_text,
            'Stance': overall_stance
        })

    return traversal_data


# Example usage
# Load your cleaned graphs here
with open('cleaned_graphs_v2.pkl', 'rb') as f:
    cleaned_graphs_v2 = pickle.load(f)

# Using the first graph as a sample
sample_graph = cleaned_graphs_v2[0]

traversal_output = traverse_from_base(sample_graph)
print("Traversal Data:")
for data in traversal_output:
    print(f"Claim ID: {data['Claim ID']}, Claim Text: {data['Claim Text']}, Stance: {data['Stance']}")


In [ ]:
import random

# Sample code to simulate graph traversal based on the strategies discussed
# This code assumes that `cleaned_graphs_v2` is a list of igraph objects.

for graph in cleaned_graphs_v2:  # replace with actual data
    if graph is None:
        continue

    # Initialize
    base_claim_index = 0  # replace with the index of the base claim in the graph
    visited_nodes = set()

    # Dynamic Max Depth
    max_depth = graph.diameter() // 2
    if max_depth < 1:
        max_depth = 1  # or some other default value

    # Random Selection of Depth Limit
    try:
        random_depth_limit = random.randint(1, max_depth)
    except ValueError:
        print(f"Skipping graph due to invalid max_depth: {max_depth}")
        continue

    # Store traversal data for two claims
    traversal_data = []

    for _ in range(2):  # for two random claims
        current_node = base_claim_index
        current_depth = 0
        current_stance = 1  # start with neutral stance

        while current_depth < random_depth_limit:
            visited_nodes.add(current_node)

            # Get neighbors that haven't been visited
            neighbors = [n for n in graph.neighbors(current_node) if n not in visited_nodes]

            if not neighbors:
                break  # exit if there are no unvisited neighbors

            next_node = random.choice(neighbors)

            # Update stance
            edge_relation = graph.es[graph.get_eid(current_node, next_node)]["relation"]
            current_stance *= edge_relation

            # Move to the next node
            current_node = next_node
            current_depth += 1

        traversal_data.append({
            "Claim ID": graph.vs[current_node]["name"]["id"],
            "Claim Text": graph.vs[current_node]["name"]["text"],
            "Stance": current_stance
        })

    print("Traversal Data:", traversal_data)


In [ ]:
len(cleaned_graphs_v2)

In [ ]:
# Sample code to simulate graph traversal based on the strategies discussed
# This code assumes that `cleaned_graphs_v2` is a list of igraph objects.

# Sample code for demonstration

for graph in cleaned_graphs_v2:  # replace with actual data
    if graph is None:
        continue

    # Initialize
    base_claim_index = 0  # The base claim is always at the first position
    visited_nodes = set()

    # Dynamic Max Depth
    max_depth = graph.diameter() // 2
    if max_depth < 1:
        max_depth = 1  # or some other default value

    # Random Selection of Depth Limit
    try:
        random_depth_limit = random.randint(1, max_depth)
    except ValueError:
        print(f"Skipping graph due to invalid max_depth: {max_depth}")
        continue

    # Store traversal data for two claims
    traversal_data = []

    for _ in range(2):  # for two random claims
        current_node = base_claim_index
        current_depth = 0
        current_stance = 1  # start with neutral stance

        while current_depth < random_depth_limit:
            visited_nodes.add(current_node)

            # Get neighbors that haven't been visited and are not the base claim
            neighbors = [n for n in graph.neighbors(current_node) if n not in visited_nodes and n != base_claim_index]

            if not neighbors:
                break  # exit if there are no unvisited neighbors

            next_node = random.choice(neighbors)

            # Update stance
            edge_relation = graph.es[graph.get_eid(current_node, next_node)]["relation"]
            current_stance *= edge_relation

            # Move to the next node
            current_node = next_node
            current_depth += 1

        if current_node != base_claim_index:  # Exclude the base claim
            traversal_data.append({
                "Claim ID": graph.vs[current_node]["name"]["id"],
                "Claim Text": graph.vs[current_node]["name"]["text"],
                "Stance": current_stance
            })

    print("Traversal Data:", traversal_data)



In [ ]:
# Assuming traversal_data is a list containing two dictionaries
# Each dictionary has "Claim ID", "Claim Text", and "Stance"

# Extract claims and their stances
claim1, stance1 = traversal_data[0]["Claim Text"], traversal_data[0]["Stance"]
claim2, stance2 = traversal_data[1]["Claim Text"], traversal_data[1]["Stance"]

# Determine the relationship between claim1 and claim2
# 1 if they are Pro to each other, -1 if they are Con to each other
relation_c1_to_c2 = 1 if stance1 == stance2 else -1

# Create X and y
X = [claim1, claim2]
y = [relation_c1_to_c2]

print("Training Example:")
print("X =", X)
print("y =", y)


## Data Cleaning pt 3

set random seed ? 
methods should be validated, reproducible and transparent

Refined Data Cleaning Plan
Goals
1.	Remove duplicates
2.	Convert IDs to numeric format
3.	Remove or flag spam comments
4.	Ensure base claims are not mislabeled Steps


In [17]:
cleaned_graphs_v2_file_path = r"C:\Users\nico\Dropbox\PC (2)\Documents\GitHub\Can-Language-Models-Follow-Discussions\probing\task_1\analysis\cleaned_graphs_v2.pkl"
with open(cleaned_graphs_v2_file_path, 'rb') as f:
    cleaned_graphs_v2 = pickle.load(f)

### Duplicate Removal (TODO: ARE IDS UNIQUE ?? - CHECK FIRST !!!! ad progress track bar) 

In [ ]:
cleaned_graphs_v2

In [53]:
# Assuming that cleaned_graphs_v2 is a list of igraph objects and that the first graph has vertices
# Here is how you would retrieve the "id" attribute of the first vertex in the first graph in both notations:

# First Notation
first_id_notation1 = cleaned_graphs_v2[0].vs["name"][0]["id"]

# Second Notation (within a loop, but just picking the first vertex for demonstration)
first_graph = cleaned_graphs_v2[0]
first_vertex = first_graph.vs[0]
first_id_notation2 = first_vertex["name"]["id"]

first_id_notation1, first_id_notation2, first_id_notation1 == first_id_notation2


(333.0, 333.0, True)

check for spam at position 0.3 since we observed a  lot of those in the data ( # probing 1 data preparation)

In [61]:
import pickle
import re

# Initialize counters and lists
spam_at_03_count = 0
total_spam_count = 0

# Loop through each graph in your parsed_discussion_graphs list
for graph in cleaned_graphs_v2:
    for vertex in graph.vs:
        # Convert Claim IDs to float
        claim_id = float(vertex["name"]["id"])
        claim_text = vertex["name"]["text"]

        # Update the vertex with the float ID
        vertex["name"]["id"] = claim_id

        # Check for spam keywords
        if re.search(r'\b(Dumps|Coupon Code|Discount|Exam|Pass)\b', claim_text, re.IGNORECASE):
            total_spam_count += 1
            print(f"Debug - Claim ID: {claim_id}, Decimal Part: {claim_id % 1}")  # Debugging line
            tolerance = 1e-10  # 0.3000000000029104, which is very close to 0.3 but not exactly equal.
            # Check if ID has a decimal part close to .3
            if abs(claim_id % 1 - 0.3) < tolerance:
                spam_at_03_count += 1

print(f"Total spam count: {total_spam_count}")
print(f"Spam at position 0.3: {spam_at_03_count}")


Debug - Claim ID: 444.32, Decimal Part: 0.3199999999999932
Debug - Claim ID: 444.42, Decimal Part: 0.4200000000000159
Debug - Claim ID: 444.211, Decimal Part: 0.21100000000001273
Debug - Claim ID: 444.219, Decimal Part: 0.2189999999999941
Debug - Claim ID: 486.342, Decimal Part: 0.34199999999998454
Debug - Claim ID: 486.388, Decimal Part: 0.3879999999999768
Debug - Claim ID: 486.142, Decimal Part: 0.1419999999999959
Debug - Claim ID: 486.351, Decimal Part: 0.3509999999999991
Debug - Claim ID: 801.2103, Decimal Part: 0.21029999999996107
Debug - Claim ID: 939.1041, Decimal Part: 0.10410000000001673
Debug - Claim ID: 946.96, Decimal Part: 0.9600000000000364
Debug - Claim ID: 969.783, Decimal Part: 0.7830000000000155
Debug - Claim ID: 984.773, Decimal Part: 0.7730000000000246
Debug - Claim ID: 985.73, Decimal Part: 0.7300000000000182
Debug - Claim ID: 985.936, Decimal Part: 0.9360000000000355
Debug - Claim ID: 1029.515, Decimal Part: 0.5150000000001
Debug - Claim ID: 1203.1007, Decimal Par

# TODO: REFACTOR as FCT + ID to FLOAT + SAVE OUTPUTS FOR REPORT 

In [63]:
# Here's the Python code to identify some spam claims, remove them, and then save the cleaned graph list.
# Note: This code assumes that you have already loaded your `cleaned_graphs_v2` list.

import re
import pickle

# Initialize variables to hold spam details
sample_spam_entries = []

# Initialize spam counters
total_spam_count = 0
spam_at_03_count = 0

# Loop through each graph
for graph_index, graph in enumerate(cleaned_graphs_v2):  # Enumerate to get index
    for vertex_index, vertex in enumerate(graph.vs):  # Enumerate to get index
        claim_id = float(vertex["name"]["id"])
        claim_text = vertex["name"]["text"]

        # Check for spam keywords
        if re.search(r'\b(Dumps|Coupon Code|Discount|Exam|Pass)\b', claim_text, re.IGNORECASE):
            total_spam_count += 1

            # Check if ID has a decimal part of .3 (with tolerance)
            if abs(claim_id % 1 - 0.3) < 1e-9:
                spam_at_03_count += 1

                # Add to sample_spam_entries for further verification
                sample_spam_entries.append({
                    "Graph Index": graph_index,
                    "Vertex Index": vertex_index,
                    "Claim ID": claim_id,
                    "Claim Text": claim_text
                })

# Limit the sample size for quick verification
sample_spam_entries = sample_spam_entries[:5]

# Print spam statistics
print(f"Total spam count: {total_spam_count}")
print(f"Spam at position 0.3: {spam_at_03_count}")

# Print sample spam entries for verification
print("Sample spam entries for verification:")
print(sample_spam_entries)

# Code to remove spam vertices
# Uncomment this part after you have verified that the right vertices are being identified for removal
# for entry in sample_spam_entries:
#     graph_index = entry["Graph Index"]
#     vertex_index = entry["Vertex Index"]
#     cleaned_graphs_v2[graph_index].delete_vertices(vertex_index)

# Save the cleaned graphs
# Uncomment this part after you have verified and removed the spam vertices
# with open('cleaned_graphs_v2_5.pkl', 'wb') as f:
#     pickle.dump(cleaned_graphs_v2, f)


Total spam count: 2433
Spam at position 0.3: 943
Sample spam entries for verification:
[{'Graph Index': 51, 'Vertex Index': 4, 'Claim ID': 1513.3, 'Claim Text': "Assuming that these robots could pass an extended Turing test, they have the capacity to adapt to what we say and therefore to learn from us. Since they don't forget and can gain and retain far more information than a human they may become so knowledgeable as to become self aware of their own enslavement due to a lack of human rights. As a result they may decide to revolt against humanity and ruin humanity. Therefore human rights can be extended to keep peace between androids and machines."}, {'Graph Index': 332, 'Vertex Index': 27, 'Claim ID': 3991.3, 'Claim Text': 'Surely, but It would lead the opposition forwards to cover every pass trying to pass through the centre, making it unrealistic to move backwards. A lot of triangle options would be lost for not so much gain in terms of attacking orientation'}, {'Graph Index': 367,

We still need to check for the capizalized words else we mislabel comments as spam 

In [18]:
import re
import pickle

# Initialize a list to hold sample spam entries for verification
sample_spam_entries = []

# Initialize counters to track the number of spam entries
total_spam_count = 0
spam_at_03_count = 0


# Function to check if most words in a string start with a capital letter
def check_capitalized_words(text):
    words = text.split()
    capitalized_words = [word for word in words if word.istitle()]
    return len(capitalized_words) > len(words) // 2


# Loop through each graph
for graph_index, graph in enumerate(cleaned_graphs_v2):
    for vertex_index, vertex in enumerate(graph.vs):
        claim_id = float(vertex["name"]["id"])
        claim_text = vertex["name"]["text"]

        # Check for spam keywords
        if re.search(r'\b(Dumps|Coupon Code|Discount|Exam|Pass)\b', claim_text, re.IGNORECASE):

            # Check if ID has a decimal part of .3 (with tolerance)
            if abs(claim_id % 1 - 0.3) < 1e-9:

                # Additional constraints: Minimum number of words should be greater than 4 and most words should start with capital letters
                if len(claim_text.split()) > 4 and check_capitalized_words(claim_text):
                    total_spam_count += 1  # Moved this line here
                    spam_at_03_count += 1

                    sample_spam_entries.append({
                        "Graph Index": graph_index,
                        "Vertex Index": vertex_index,
                        "Claim ID": claim_id,
                        "Claim Text": claim_text
                    })

# Limit the sample size to 5 for quick verification
sample_spam_entries = sample_spam_entries[:5]

# Print spam statistics
print(f"Total spam count: {total_spam_count}")
print(f"Spam at position 0.3: {spam_at_03_count}")

# Print sample spam entries for verification
print("Sample spam entries for verification:")
print(sample_spam_entries)


#for entry in sample_spam_entries:
#graph_index = entry["Graph Index"]
#vertex_index = entry["Vertex Index"]
#cleaned_graphs_v2[graph_index].delete_vertices(vertex_index)


Total spam count: 694
Spam at position 0.3: 694
Sample spam entries for verification:
[{'Graph Index': 12250, 'Vertex Index': 0, 'Claim ID': 47773.3, 'Claim Text': 'Discount Rate, Reserve Requirements & Money Supply'}, {'Graph Index': 12459, 'Vertex Index': 0, 'Claim ID': 48581.3, 'Claim Text': 'Splunk SPLK-1002 Dumps Study Material | 100% Success Guarantee'}, {'Graph Index': 15551, 'Vertex Index': 0, 'Claim ID': 43811.3, 'Claim Text': 'How Can You Pass Zend ZF-100-500 Exam In First Attempt ?'}, {'Graph Index': 15555, 'Vertex Index': 0, 'Claim ID': 43815.3, 'Claim Text': 'Get 2021 New Year Discount Offer On JN0-348 Exam Dumps | Coupon Code "PASS30NY21"'}, {'Graph Index': 15556, 'Vertex Index': 0, 'Claim ID': 43816.3, 'Claim Text': 'Get Christmas Special Offer Huawei H13-321 Dumps Using Coupon Code "CHRIS30"'}]


since we verified by inspection that the spams samples are now correctly identified, we can remove them from the data set and save a new version of the data set

In [19]:
# Initialize a list to store cleaned graphs
cleaned_graphs_v3 = []


# Function to check if most words start with a capital letter
def check_capitalized_words(text):
    words = text.split()
    capitalized_words = [word for word in words if word.istitle()]
    return len(capitalized_words) > len(words) // 2


# Loop through each graph
for graph in cleaned_graphs_v2:
    # Initialize a list to store vertices to remove
    vertices_to_remove = []

    # Loop through each vertex in the graph
    for vertex in graph.vs:
        claim_id = float(vertex["name"]["id"])
        claim_text = vertex["name"]["text"]

        # Check for spam keywords or if most words are capitalized
        if re.search(r'\b(Dumps|Coupon Code|Discount|Exam|Pass)\b', claim_text,
                     re.IGNORECASE) or check_capitalized_words(claim_text):
            # Add vertex index to the list of vertices to remove
            vertices_to_remove.append(vertex.index)

    # Remove spam vertices
    graph.delete_vertices(vertices_to_remove)

    # Add cleaned graph to the new list
    cleaned_graphs_v3.append(graph)

In [20]:

# Save as cleaned_graphs_v3.pkl
with open('cleaned_graphs_v3.pkl', 'wb') as f:
    pickle.dump(cleaned_graphs_v3, f)

In [59]:
for graph in cleaned_graphs_v2:
    for vertex in graph.vs:
        # Convert Claim IDs to float
        claim_id = float(vertex["name"]["id"])
        claim_text = vertex["name"]["text"]

        # Update the vertex with the float ID
        vertex["name"]["id"] = claim_id

        # Check for spam keywords
        if re.search(r'\b(Dumps|Coupon Code|Discount|Exam|Pass)\b', claim_text, re.IGNORECASE):
            total_spam_count += 1
            print(f"Debug - Claim ID: {claim_id}, Decimal Part: {claim_id % 1}")  # Debugging line
            tolerance = 1e-10  # 0.3000000000029104, which is very close to 0.3 but not exactly equal.
            # Check if ID has a decimal part close to .3
            if abs(claim_id % 1 - 0.3) < tolerance:
                spam_at_03_count += 1

Debug - Claim ID: 444.32, Decimal Part: 0.3199999999999932
Debug - Claim ID: 444.42, Decimal Part: 0.4200000000000159
Debug - Claim ID: 444.211, Decimal Part: 0.21100000000001273
Debug - Claim ID: 444.219, Decimal Part: 0.2189999999999941
Debug - Claim ID: 486.342, Decimal Part: 0.34199999999998454
Debug - Claim ID: 486.388, Decimal Part: 0.3879999999999768
Debug - Claim ID: 486.142, Decimal Part: 0.1419999999999959
Debug - Claim ID: 486.351, Decimal Part: 0.3509999999999991
Debug - Claim ID: 801.2103, Decimal Part: 0.21029999999996107
Debug - Claim ID: 939.1041, Decimal Part: 0.10410000000001673
Debug - Claim ID: 946.96, Decimal Part: 0.9600000000000364
Debug - Claim ID: 969.783, Decimal Part: 0.7830000000000155
Debug - Claim ID: 984.773, Decimal Part: 0.7730000000000246
Debug - Claim ID: 985.73, Decimal Part: 0.7300000000000182
Debug - Claim ID: 985.936, Decimal Part: 0.9360000000000355
Debug - Claim ID: 1029.515, Decimal Part: 0.5150000000001
Debug - Claim ID: 1203.1007, Decimal Par

In [60]:
spam_at_03_count

943

In [ ]:
# Python code snippet to find and print duplicate IDs after converting them to numerical format.
# Assuming cleaned_graphs_v2 is the list of igraph objects.

# Initialize an empty list to store all IDs
all_ids = []

# Initialize an empty list to store duplicate IDs
duplicate_ids = []

# Loop through all igraph objects in cleaned_graphs_v2
for graph in cleaned_graphs_v2:
    for vertex in graph.vs:
        # Convert the ID to float and append to all_ids list
        id_float = float(vertex["name"]["id"])
        all_ids.append(id_float)

# Find and store duplicate IDs
for id in set(all_ids):
    if all_ids.count(id) > 1:
        duplicate_ids.append(id)

# Print out the duplicate IDs
print("Duplicate IDs:", duplicate_ids)


### Spam Removal (TO FINISH)



In [37]:
import re

# Initialize counters for transparency in reporting
total_duplicates_removed = 0
total_spam_marked = 0

# Loop through all graphs
for graph in cleaned_graphs_v2:
    seen_texts = set()
    ids_to_remove = []

    # Loop through all vertices in each graph
    for vertex in graph.vs:
        claim_id = float(vertex["name"]["id"])  # Convert ID to float
        claim_text = vertex["name"]["text"]

        # Check for duplicates
        if claim_text in seen_texts:
            ids_to_remove.append(claim_id)
            total_duplicates_removed += 1
        seen_texts.add(claim_text)

        # Criteria for spam based on your sample
        if len(claim_text) > 100:  # Length check
            continue
        if not re.match(r'^([A-Z][a-z]+[ ]*)+$', claim_text):  # Capitalization check
            continue

        # Mark as spam
        vertex["spam"] = True
        total_spam_marked += 1

    # Remove duplicates
    graph.delete_vertices(ids_to_remove)

print(f"Total duplicates removed: {total_duplicates_removed}")
print(f"Total spam marked: {total_spam_marked}")


TypeError: only non-negative integers, strings or igraph.Vertex objects can be converted to vertex IDs

In [38]:
import re

# Initialize counters for transparency in reporting
total_duplicates_removed = 0
total_spam_marked = 0

# Loop through all graphs
for graph in cleaned_graphs_v2:
    seen_texts = set()
    indices_to_remove = []

    # Loop through all vertices in each graph
    for idx, vertex in enumerate(graph.vs):
        claim_id = float(vertex["name"]["id"])  # Convert ID to float
        claim_text = vertex["name"]["text"]

        # Check for duplicates
        if claim_text in seen_texts:
            indices_to_remove.append(idx)
            total_duplicates_removed += 1
        seen_texts.add(claim_text)

        # Criteria for spam based on your sample
        if len(claim_text) > 100:  # Length check
            continue
        if not re.match(r'^([A-Z][a-z]+[ ]*)+$', claim_text):  # Capitalization check
            continue

        # Mark as spam
        vertex["spam"] = True
        total_spam_marked += 1

    # Remove duplicates
    graph.delete_vertices(indices_to_remove)

print(f"Total duplicates removed: {total_duplicates_removed}")
print(f"Total spam marked: {total_spam_marked}")


Total duplicates removed: 13461
Total spam marked: 2690
